In [2]:
from fastai.vision.all import *

In [3]:
PATH = Path('../input/optiver-realized-volatility-prediction')

# data_df = pd.read_parquet(PATH/'book_train_reindexed.parquet')

# sizes = ['bid_size1', 'bid_size2', 'ask_size1', 'ask_size2']
# for sz in sizes:
#     data_df[sz] = data_df[sz]/ 1_000_000



In [4]:
#train_df = pd.read_csv(PATH/'train_with_offset.csv')
# data_df['wap'] = (data_df.bid_price1 * data_df.ask_size1 + data_df.ask_price1 * data_df.bid_size1) / (data_df.ask_size1 + data_df.bid_size1)

# mean_wap =[]
# for row in train_df.itertuples():
#     df = data_df.iloc[row.offset:row.offset+600]
#     mean_wap.append(df.wap.mean())

In [5]:
# train_df['mean_wap']=mean_wap
# train_df.to_csv(PATH/'train_with_wap.csv')

In [6]:
# np_data = data_df[['bid_price1', 'ask_price1', 'bid_size1', 'ask_size1','bid_price2', 'ask_price2', 'bid_size2', 'ask_size2']].to_numpy()

# del data_df

# np_data = np_data.astype('float32')
# torch_data = torch.tensor(np_data)

# del np_data

# torch.save(torch_data, PATH/'torch_data.pth')

In [ ]:
%%time
torch_data = torch.load(PATH/'torch_data.pth')

In [8]:
#torch_data[:,[2,3,6,7]] = 1000000 * torch_data[:,[2,3,6,7]] 

In [1]:
means, stds = torch_data.mean(dim=0), torch_data.std(dim=0)
print(means, stds)
torch_data = (torch_data - means) / stds

NameError: name 'torch_data' is not defined

In [ ]:
train_df = pd.read_csv(PATH/'train_with_wap.csv')

In [ ]:
w = train_df.mean_wap.to_numpy()

w =np.sort(w)

bins = []

step = (len(w)+9)//10
for i in range(0, len(w), step):
    j = min(i+step, len(w))
    bins.append(w[j] if j< len(w) else 2)
    print(w[i], w[j-1],np.median(w[i:j]) )

wap_bin = np.digitize(train_df.mean_wap, bins)

train_df['wap_bin']=wap_bin

In [ ]:
w = train_df.target.to_numpy()

w =np.sort(w)

bins = []
bin_med=[]
step = (len(w)+9)//10
for i in range(0, len(w), step):
    j = min(i+step, len(w))
    bins.append(w[j] if j< len(w) else 1)
    bin_med.append(np.median(w[i:j]))
    print(w[i], w[j-1],np.median(w[i:j]) )

target_bin = np.digitize(train_df.target, bins)

train_df['target_bin']=target_bin

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, ch):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv1d(ch, ch, kernel_size = 5, padding = 2, padding_mode='replicate'),
            nn.BatchNorm1d(ch),
            nn.ReLU(),
            nn.Conv1d(ch, ch, kernel_size = 5, padding = 2, padding_mode='replicate'),
            nn.BatchNorm1d(ch),
        )
        
    def forward(self, x):
        res = self.layers(x) + x
        res = F.relu(res)
        return res

In [ ]:
class ResnetModel(nn.Module):
    def __init__(self, num_outputs, do_sigmoid = False):
        super().__init__()
        self.do_sigmoid = do_sigmoid
        chan = 20
        layers = [nn.Conv1d(8, chan, kernel_size=1)]
        
        for _ in range(6):
            layers += [ResBlock(chan)
                       , nn.AvgPool1d(3, padding=1),
                      ]
        layers += [Flatten(), nn.Dropout(.1)]   
        self.conv_layers = nn.Sequential(*layers)
        self.classifier = nn.Linear(chan, num_outputs)
        
    def forward(self, x):
        feat = self.conv_layers(x)
        res = self.classifier(feat)
        if self.do_sigmoid:
            res = sigmoid_range(res, 0, .1).view(-1)
        return res

In [ ]:
def stock_id(row):
    return row.stock_id

In [ ]:
kfold = GroupKFold()
trn_idx, val_idx = first(GroupKFold().split(train_df, groups = train_df.time_id))

In [ ]:
class LoadTransform(Transform):
    def __init__(self, torch_data):
        super().__init__()
        self.data = torch_data
        
    def encodes(self, row):
        ret = self.data[row.offset:row.offset+600].T
        #return ret.view(1,600,8)
        return ret

tfm = LoadTransform(torch_data)

data_block = DataBlock(
        blocks = (TransformBlock(type_tfms=tfm), 
                  CategoryBlock(vocab=list(range(10)))),
        splitter=IndexSplitter(val_idx),
        get_y=ColReader('target_bin'),
)
dls = data_block.dataloaders(train_df, bs=1024)

In [61]:
dls_reg = DataBlock(
        blocks = (TransformBlock(type_tfms=tfm), 
                  #TransformBlock(type_tfms=stock_id),
                  RegressionBlock()),
        splitter=ColSplitter(),
        get_y=ColReader('target'),
).dataloaders(train_df, bs = 512)


In [62]:
bx,by = dls.one_batch()

In [63]:
md = ResnetModel(10).cuda()
md(bx).shape

torch.Size([1024, 10])

In [64]:
def rmspe(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [65]:
def rmspe_loss(preds, targs, reduction='mean'):
    assert reduction == 'mean'
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [66]:
#learn = Learner(dls, ConvModel(),loss_func=mspe_loss, metrics=AccumMetric(rmspe))
learn = Learner(dls, ResnetModel(10), metrics = [accuracy], cbs=[MixUp])

In [53]:
learn.fit_one_cycle(12,1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.993149,1.764510,0.333472,00:29
1,1.816978,1.563564,0.374848,00:29
2,1.786999,1.552353,0.383899,00:29
3,1.783467,1.562419,0.369468,00:29
4,1.764555,1.610426,0.357795,00:29
5,1.762451,1.489208,0.405928,00:29
6,1.755247,1.515467,0.405123,00:58
7,1.748991,1.531275,0.397560,01:56
8,1.745338,1.501794,0.404062,00:46
9,1.740241,1.480284,0.411369,00:58


In [54]:
torch.save(learn.model, 'models/conv_model.pth')

Path('models/res_model2.pth.pth')

In [74]:

reg_m = ResnetModel(1, do_sigmoid=True)
reg_m.conv_layers = torch.load('models/conv_model.pth').conv_layers


In [75]:
def split_2way(model):
    #return L(params(model.initial_conv)+params(model.conv_layers), params(model.classifier))
    return L(params(model.conv_layers), params(model.classifier))

In [76]:
learn2 = Learner(dls_reg, reg_m, splitter=split_2way, loss_func=rmspe_loss, metrics=AccumMetric(rmspe))

In [77]:
learn2.fine_tune(20, 5e-3,freeze_epochs=3)

epoch,train_loss,valid_loss,rmspe,time
0,0.931246,0.952462,0.981700,00:34
1,0.482036,0.328781,0.334759,00:22
2,0.296890,0.251767,0.256399,00:23


epoch,train_loss,valid_loss,rmspe,time
0,0.277516,0.250693,0.254660,00:23
1,0.268144,0.247572,0.252382,00:23
2,0.264880,0.271888,0.277059,00:23
3,0.257460,0.250145,0.255909,00:23
4,0.257397,0.254583,0.259265,00:23
5,0.255829,0.267972,0.272393,00:23
6,0.253897,0.261224,0.265723,00:23
7,0.252094,0.253924,0.259239,00:23
8,0.250492,0.257383,0.261833,00:23
9,0.249022,0.258480,0.265542,00:27
